# What are the details (metadata) of one of my _files_?
### Overview
Here we focus on _getting the details_ (or _metadata_) from a single file. As with any **detail**-type call, we will get extensive information about one file but must first know that files's id. 
 
### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="files_listAll.ipynb" target="_blank">list</a> files within one of your projects (we will just use that call directly here).
 5. Your project needs to have at least one file inside
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# User input: specify platform {cgc, sbg}
prof = 'cgc'

config_config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_config_file)

## List my files and detail the first one
We start by listing all of your files, then get more information on the first one. A **detail**-call for files returns the following *attributes*:
* **created_on** File creation date
* **id**     _Unique_ identifier for the file
* **name**   Name of the file, note this **is** metadata and can be _changed_
* **href**   Address<sup>1</sup> of the file.
* **modified_on** File modification date
* **metadata** Dictionary of metadata
* **origin**  Will link back to a *task* if this is an output file 
* **project** Project the file is in
* **size** file size in bytes

The **metadata** dictionary contains all metadata properties. Note there are **SUBSTANTIAL DIFFERENCES** here between _TCGA_ and _uploaded_ files. For TCGA files, metadata is **immutable** and the list is:
* vital_status
* reference_genome
* aliquot_id
* age_at_diagnosis
* sample_id
* days_to_death
* ethnicity
* sample_uuid
* sample_type
* platform
* data_subtype
* experimental_strategy
* case_uuid
* data_type
* case_id
* investigation
* aliquot_uuid
* disease_type
* gender
* data_format
* race
* primary_site

In contrast the metadata is **mutable** for user uploaded files.

<sup>1</sup> This is the address where, by using API you can get this resource

In [ ]:
# [USER INPUT] Set project name and file (f_) indices here:
project_name = 'Life is Beautiful'                          
f_index = 0


# LIST all projects and check for name match
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == project_name]   

if not my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('The project (%s) does not exist, please check spelling (especially trailing spaces)' \
          % project_name)
    raise KeyboardInterrupt
else:
    # list the files in the target project
    my_files = [f for f in api.files.query(my_project[0].id).all()]
    # get details of the first file inside the project
    single_file = api.files.get(id = my_files[f_index].id)
    print('You have selected file %s (size %s [bytes]). \n' % (single_file.name, single_file.size))

    # TODO: work on this for SBPLAT compatibility
    if hasattr(single_file, 'metadata') and single_file.metadata is not None:
        if ('age_at_diagnosis' in single_file.metadata.keys()) \
        and ('experimental_strategy' in single_file.metadata.keys()):
            print(('Subject was %i years old at diagnosis; alliquot was processed with %s' % \
                   (single_file.metadata['age_at_diagnosis'], \
                    single_file.metadata['experimental_strategy'] ))) 

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/get-file-details)